In [61]:
from langchain_ollama  import OllamaLLM
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
import pandas as pd
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer
from app.utils.functions import *
from app.core import config

In [170]:
def preprocessing_data(df: pd.DataFrame, simple_dict: list[dict]) -> pd.DataFrame:
    """Prétraite et catégorise le DataFrame selon les groupes définis."""
    try:
        required_columns = ['Lignes', 'Contexte', 'Nature de l\'écriture', 'Année', 'Mois', 'Montant']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"La colonne requise '{col}' est absente du DataFrame.")
        mask = (
            (df.iloc[:, 0] == df.iloc[0, 0]) &
            (df.iloc[:, 2] == "Compte d'exploitation") &
            (df.iloc[:, 8] != "Colonne variation")
        )
        df_filtered: pd.DataFrame
        df_filtered = df[mask].copy()
        
        if df_filtered.empty:
            df_filtered['Groupe'] = pd.Series(dtype='object')
            return df_filtered

        mask_pct_recettes = df_filtered['Lignes'] == "% DES RECETTES TOTALES"
        df_pct_recettes = df_filtered[mask_pct_recettes].copy()
        df_autres = df_filtered[~mask_pct_recettes].copy()

        if not df_pct_recettes.empty:
            sort_keys = ['Année', 'Mois', 'Contexte', "Nature de l'écriture", 'Montant']
            existing_sort_keys = [col for col in sort_keys if col in df_pct_recettes.columns]
            df_pct_recettes = (
                df_pct_recettes
                .sort_values(by=existing_sort_keys[:-1] + ['Montant'], ascending=[True]*len(existing_sort_keys[:-1])+[False])
                .reset_index(drop=True)
            )
            code_map = {0: "4.", 1: "7.", 2: "10."}
            df_pct_recettes['Code Hiérarchique'] = (
                df_pct_recettes
                .groupby(existing_sort_keys[:-1], sort=False)
                .cumcount()
                .map(lambda idx: code_map.get(idx, None))
            )

        coded_tree = generate_hierarchy_codes(simple_dict)
        hierarchy_list = extract_flat_hierarchy_list(coded_tree)
        mapping_dict = {item['label']: item['code'] for item in hierarchy_list}

        if not df_autres.empty:
            df_autres['Code Hiérarchique'] = df_autres['Lignes'].map(mapping_dict)
        df_filtered = pd.concat([df_autres, df_pct_recettes]).sort_index(kind="stable")
        cols = list(df.columns)
        if "Code Hiérarchique" in cols and "Lignes" in cols:
            cols.remove("Code Hiérarchique")
            insert_idx = cols.index("Lignes")
            cols = cols[:insert_idx] + ["Code Hiérarchique"] + cols[insert_idx:]
            df_filtered = df_filtered[cols]

        result_list = extract_all_descendants_for_list(simple_dict)
        CHIFFRE_AFFAIRES = result_list[0] + result_list[3] + result_list[6] + result_list[9] + result_list[10]
        CHARGES = result_list[1] + result_list[4] + result_list[7] + result_list[12]
        MARGES = result_list[2] + result_list[5] + result_list[8] + result_list[11] + result_list[13]
        groupes_dict = {
            "Chiffre d'affaire": CHIFFRE_AFFAIRES,
            "Charge": CHARGES,
            "Marge": MARGES
        }
        groupes_mapping = {poste: groupe for groupe, postes in groupes_dict.items() for poste in postes}
        df_filtered['Groupe'] = df_filtered['Lignes'].map(groupes_mapping)

        mask_pct = df_filtered['Lignes'] == "% DES RECETTES TOTALES"
        mask_non_pct = ~mask_pct

        # Process non-pct as before
        df_filtered.loc[mask_non_pct, "Montant"] = df_filtered.loc[mask_non_pct, "Montant"].round(0).astype(int)

        # Ensure we handle dtype for mask_pct
        if mask_pct.any():
            montant_pct = df_filtered.loc[mask_pct, "Montant"].round(2).astype(str) + "%"
            # Cast to object dtype before assignment to avoid FutureWarning
            df_filtered["Montant"] = df_filtered["Montant"].astype("object")
            df_filtered.loc[mask_pct, "Montant"] = montant_pct
        
        df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)
        return df_filtered
    except Exception as e:
        logger.error(f"Erreur lors du prétraitement des données : {e}")
        raise

In [171]:
# Chargement du fichier (en supposant la même structure que précédemment)
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)
lexiques = await get_mapping()


df = pd.read_csv(r"data.csv")
df = preprocessing_data(df, simple_dict)

# Renommage et nettoyage
df = df.rename(
    columns={
        'Code Hiérarchique': 'Code_H', 
        'Montant': 'Montant',
        'Lignes': 'Ligne_Analytique',
        'Contexte': 'Contexte',
        'Année': 'Annee',
        'Groupe': 'Groupe',
        'Section  analytique': 'Residence'
    }
)

df['Annee'] = df['Annee'].astype(int)
df['Mois'] = df['Mois'].astype(int)
df['Contexte'] = df['Contexte'].replace({'R': 'Réel', 'B': 'Budget', 'P': 'Prévision'})

df_agg = df.groupby(
    [
        'Residence', 'Colonnes', 'Annee', 'Mois', "Nature de l'écriture", 'Contexte', 'Code_H', 'Ligne_Analytique', 'Groupe'
    ]
)['Montant'].sum().reset_index()

In [268]:
contexte_order = ['Réel', 'Prévision', 'Budget']

# Ajoute 'Nature de l\'écriture' dans les colonnes du pivot (et pas dans l'index)
df_pivot = df_agg.pivot_table(
    index=['Groupe', 'Code_H', 'Ligne_Analytique'],
    columns=['Annee', 'Contexte', 'Mois', "Nature de l'écriture"],
    values='Montant',
    fill_value=0,
    aggfunc='sum'
)

# Trie les colonnes en tenant compte de l'ordre contexte & du reste
if df_pivot.columns.nlevels == 4:
    cols = [
        col for col in sorted(
            df_pivot.columns,
            key=lambda x: (
                x[0],  # Annee
                x[1],  # Contexte
                contexte_order.index(x[2]) if x[2] in contexte_order else 99,  # Mois (ordre du contexte)
                x[3]   # Nature de l'écriture
            )
        )
    ]
    df_pivot = df_pivot[cols]

df_pivot = df_pivot.reset_index()
df_pivot

Annee                 Groupe   Code_H                   Ligne_Analytique  \
Contexte                                                                   
Mois                                                                       
Nature de l'écriture                                                       
0                     Charge      13.                              CAPEX   
1                     Charge       2.        CHARGES D'IMMEUBLE DIRECTES   
2                     Charge     2.1.                 FRAIS DE PERSONNEL   
3                     Charge   2.1.1.                           Salaires   
4                     Charge  2.1.10.  Prestataires sécurité gardiennage   
..                       ...      ...                                ...   
146                    Marge      12.                             EBITDA   
147                    Marge      14.                     FREE CASH FLOW   
148                    Marge       3.                            MARGE 1   
149                    Marge       6.                            MARGE 2   
150                    Marge       9.                            MARGE 3   

Annee                      2022                                          \
Contexte                   Réel                                           
Mois                         12         1         2         3         4   
Nature de l'écriture   Annuelle Mensuelle Mensuelle Mensuelle Mensuelle   
0                       95452.0         0         0         0   16774.0   
1                      674637.0  113102.0   25643.0   95519.0   62063.0   
2                      161106.0    8556.0   10998.0   19345.0   11742.0   
3                       46528.0    4146.0    4170.0    4254.0    4633.0   
4                         -78.0    -112.0         0         0         0   
..                          ...       ...       ...       ...       ...   
146                     54004.0   39274.0  -36423.0  -53258.0   78695.0   
147                    -41447.0   39274.0  -36423.0  -53258.0   61921.0   
148                   1181378.0   53541.0  116352.0   43904.0   95627.0   
149                    178054.0   43339.0  -36559.0  -28520.0   85447.0   
150                     39216.0   38426.0  -37189.0  -54106.0   77874.0   

Annee                                     ...      2026                      \
Contexte                                  ...    Budget                       
Mois                         5         6  ...         3         4         5   
Nature de l'écriture Mensuelle Mensuelle  ... Mensuelle Mensuelle Mensuelle   
0                       5431.0         0  ...         0         0         0   
1                      23530.0   43562.0  ...   50173.0   57354.0   25080.0   
2                      11108.0   14427.0  ...   17760.0   12274.0    9754.0   
3                       3908.0    4698.0  ...    6677.0    6677.0    6677.0   
4                            0         0  ...         0         0         0   
..                         ...       ...  ...       ...       ...       ...   
146                   -39898.0   -6439.0  ... -131624.0  110156.0  116129.0   
147                   -45329.0   -6439.0  ... -131624.0  110156.0  116129.0   
148                   109683.0   93433.0  ...  123100.0  147603.0  146724.0   
149                   -40515.0   18937.0  ... -119578.0  128863.0  127934.0   
150                   -40760.0   -7305.0  ... -134371.0  107481.0  113382.0   

Annee                                                                   \
Contexte                                                                 
Mois                         6         7         8         9        10   
Nature de l'écriture Mensuelle Mensuelle Mensuelle Mensuelle Mensuelle   
0                            0         0         0         0         0   
1                      42573.0   63333.0   29281.0   34167.0   60469.0   
2                      12896.0   15299.0   13764.0    9544.0   10331.0   
3                       6677.

In [175]:
def code_hierarchical_sort_key(code):
    parts = str(code).strip('.').split('.')
    return [int(part) if part.isdigit() else part for part in parts if part]

df_pivot_sorted = df_pivot.copy()
df_pivot_sorted['__sort_key'] = df_pivot_sorted['Code_H'].apply(code_hierarchical_sort_key)
sorted_indexes = df_pivot_sorted.sort_values('__sort_key').index
df_pivot_sorted = df_pivot_sorted.loc[sorted_indexes].drop(columns='__sort_key').reset_index(drop=True)

df_pivot_sorted.to_csv("df_pivot.csv")
df_pivot_sorted

C:\Users\Amsata\AppData\Local\Temp\ipykernel_27036\3123885414.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_pivot_sorted = df_pivot_sorted.loc[sorted_indexes].drop(columns='__sort_key').reset_index(drop=True)


Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire      1.   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
146                   Chiffre d'affaire     10.   
147                   Chiffre d'affaire     11.   
148                               Marge     12.   
149                              Charge     13.   
150                               Marge     14.   

Annee                                     Ligne_Analytique       2022  \
Contexte                                                         Réel   
Mois                                                               12   
Nature de l'écriture                                         Annuelle   
0                                                 RECETTES  1856015.0   
1                          Loyers logements et parkings HT  1715744.0   
2                                     CA Locatif Estudines  1709962.0   
3                                      CA Locatif Parkings     5782.0   
4                                         RECETTES ANNEXES   129946.0   
..                                                     ...        ...   
146                                 % DES RECETTES TOTALES      2.11%   
147                   Dotations aux amortissements (CAPEX)    14788.0   
148                                                 EBITDA    54004.0   
149                                                  CAPEX    95452.0   
150                                         FREE CASH FLOW   -41447.0   

Annee                                                                   \
Contexte                                                                 
Mois                         1         2         3         4         5   
Nature de l'écriture Mensuelle Mensuelle Mensuelle Mensuelle Mensuelle   
0                     166643.0  141995.0  139423.0  157690.0  133213.0   
1                     143177.0  140713.0  140431.0  130839.0  130957.0   
2                     142098.0  139996.0  139868.0  130404.0  130598.0   
3                       1078.0     716.0     564.0     435.0     359.0   
4                      23467.0    1282.0      38.0   26850.0    2256.0   
..                         ...       ...       ...       ...       ...   
146                     23.06%   -26.19%   -38.81%    49.38%    -30.6%   
147                      849.0     766.0     848.0     821.0     862.0   
148                    39274.0  -36423.0  -53258.0   78695.0  -39898.0   
149                          0         0         0   16774.0    5431.0   
150                    39274.0  -36423.0  -53258.0   61921.0  -45329.0   

Annee                           ...      2026                                \
Contexte                        ...    Budget                                 
Mois                         6  ...         3         4         5         6   
Nature de l'écriture Mensuelle  ... Mensuelle Mensuelle Mensuelle Mensuelle   
0                     136995.0  ...  173274.0  204957.0  171803.0  176400.0   
1                     132729.0  ...  170690.0  168918.0  168918.0  172014.0   
2                     132427.0  ...  170090.0  168318.0  168318.0  171414.0   
3                        302.0  ...     600.0     600.0     600.0     600.0   
4                       4657.0  ...    2700.0   36150.0    3000.0    4500.0   
..                         ...  ...       ...       ...       ...       ...   
146                     -5.33%  ...   -77.55%    52.44%     66.0%    -69.2%   
147                      866.0  ...    2747.0    2675.0    2747.0    2675.0   
148                    -6439.0  ... -131624.0  110156.0  116129.0 -119385.0   


In [176]:
mask_annuelle = df_pivot_sorted.columns.get_level_values(3) == "Annuelle"
annuelle_cols = df_pivot_sorted.columns[mask_annuelle].tolist()

meta_cols = [c for c in df_pivot_sorted.columns if c[0] in ("Groupe", "Code_H", "Ligne_Analytique")]

selected_cols = meta_cols + annuelle_cols
df_pivot_sorted_annual = df_pivot_sorted.loc[:, selected_cols]

df_pivot_sorted_annual.to_csv("df_pivot_sorted_annual.csv")
df_pivot_sorted_annual

Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire      1.   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
146                   Chiffre d'affaire     10.   
147                   Chiffre d'affaire     11.   
148                               Marge     12.   
149                              Charge     13.   
150                               Marge     14.   

Annee                                     Ligne_Analytique       2022  \
Contexte                                                         Réel   
Mois                                                               12   
Nature de l'écriture                                         Annuelle   
0                                                 RECETTES  1856015.0   
1                          Loyers logements et parkings HT  1715744.0   
2                                     CA Locatif Estudines  1709962.0   
3                                      CA Locatif Parkings     5782.0   
4                                         RECETTES ANNEXES   129946.0   
..                                                     ...        ...   
146                                 % DES RECETTES TOTALES      2.11%   
147                   Dotations aux amortissements (CAPEX)    14788.0   
148                                                 EBITDA    54004.0   
149                                                  CAPEX    95452.0   
150                                         FREE CASH FLOW   -41447.0   

Annee                      2023       2024       2025                  2026  
Contexte                   Réel       Réel     Budget  Prévision     Budget  
Mois                         12         12         12         12         12  
Nature de l'écriture   Annuelle   Annuelle   Annuelle   Annuelle   Annuelle  
0                     2065453.0  2193409.0  2205868.0  2182306.0  2221683.0  
1                     1908910.0  2026364.0  2038521.0  2023357.0  2057395.0  
2                     1903970.0  2019099.0  2031296.0  2017043.0  2050195.0  
3                        4940.0     7265.0     7224.0     6315.0     7200.0  
4                      182250.0   203148.0   179398.0   171106.0   176470.0  
..                          ...        ...        ...        ...        ...  
146                      12.11%     10.22%     10.08%      8.33%      13.4%  
147                     25150.0    28095.0    27076.0    23186.0    34447.0  
148                    275367.0   252303.0   249344.0   204917.0   332103.0  
149                     34342.0    15964.0          0    19270.0          0  
150                    241025.0   236339.0   249344.0   185646.0   332103.0  

[151 rows x 9 columns]

---

In [ ]:
from typing import List, Dict, Any

# ---------------------------
# Helpers pour récupérer valeur depuis MultiIndex
# ---------------------------
def get_value_for_year(
    df_pivot: pd.DataFrame, row_idx: int, year: int,
    contexte_preference: List[str] = None,
    month: int = 12,
    nature: str = "Annuelle"
):
    """
    Récupère la valeur dans df_pivot pour la ligne `row_idx` et la colonne (year, contexte, month, nature).
    contexte_preference: ordre de préférence, ex ["Réel","Budget","Prévision"]
    Retourne (valeur, contexte_found) ou (np.nan, None)
    """
    cols = df_pivot.columns
    if contexte_preference is None:
        contexte_preference = sorted({c[1] for c in cols if not (isinstance(c, str))})
    for contexte in contexte_preference:
        target = (year, contexte, month, nature)
        if target in cols:
            return df_pivot.iloc[row_idx][target], contexte
        cand_cols = [c for c in cols if isinstance(c, tuple) and c[0] == year and c[1] == contexte and c[3] == nature]
        exact = [c for c in cand_cols if c[2] == month]
        if exact:
            return df_pivot.iloc[row_idx][exact[0]], contexte
        if cand_cols:
            return df_pivot.iloc[row_idx][cand_cols[0]], contexte
    cand = [c for c in cols if isinstance(c, tuple) and c[0] == year and c[3] == nature]
    if cand:
        return df_pivot.iloc[row_idx][cand[0]], cand[0][1]
    return np.nan, None

def safe_isfinite(x):
    import numbers
    import numpy as np
    # Only numeric types, not bool or complex
    if isinstance(x, (int, float, np.integer, np.floating)) and not isinstance(x, bool):
        return np.isfinite(x)
    else:
        return False

# ---------------------------
# Fonctions de génération de chunks
# ---------------------------
def make_line_cell_chunk(
    df_pivot: pd.DataFrame, row_idx: int, year: int,
    residence_label: str,
    contexte_preference: List[str] = ["Réel", "Prévision", "Budget"]):
    """
    Crée un chunk correspondant à la cellule annuelle pour une ligne x année (type: LINE_CELL).
    """
    row = df_pivot.iloc[row_idx]
    code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
    groupe = row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"]
    ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
    
    val, contexte_used = get_value_for_year(df_pivot, row_idx, year, contexte_preference)
    # Use safe_isfinite for robust numeric check
    if pd.notna(val) and safe_isfinite(val):
        val_str = f"{val:.0f} €"
    else:
        val_str = "N/A"
    text = (
        f"[TYPE: LINE_CELL]\n"
        f"Résidence: {residence_label}\n"
        f"Année: {year}\n"
        f"Contexte: {contexte_used or 'N/A'}\n"
        f"Groupe: {groupe}\n"
        f"Code Hiérarchique: {code_h}\n"
        f"Ligne analytique: {ligne}\n"
        f"Valeur annuelle: {val_str}\n"
    )
    metadata = {
        "chunk_type": "LINE_CELL",
        "residence": residence_label,
        "year": year,
        "contexte": contexte_used,
        "groupe": groupe,
        "code_h": code_h,
        "ligne": ligne,
        "row_index": int(df_pivot.index[row_idx])
    }
    return {"text": text, "metadata": metadata}

def make_annual_summary_chunk(
    df_pivot: pd.DataFrame, row_idx: int, year: int,
    residence_label: str,
    contexte_preference: List[str] = ["Réel", "Prévision", "Budget"]):
    """
    Synthèse textuelle d'une ligne (ligne analytique) pour une année (type: ANNUAL_SUMMARY).
    Contient valeur, variation vs previous year (si disponible), pct change if possible.
    """
    val_cur, ctx = get_value_for_year(df_pivot, row_idx, year, contexte_preference)
    val_prev, _ = get_value_for_year(df_pivot, row_idx, year-1, contexte_preference)
    # formatting
    def fmt(v):
        if pd.notna(v) and safe_isfinite(v):
            return f"{v:.0f} €"
        return "N/A"
    def pct_change(a, b):
        try:
            if pd.isna(a) or pd.isna(b) or not safe_isfinite(a) or not safe_isfinite(b) or b == 0:
                return None
            return (a - b) / abs(b) * 100
        except Exception:
            return None
    change_pct = pct_change(val_cur, val_prev)
    if (change_pct is not None and isinstance(change_pct, (int, float)) and safe_isfinite(change_pct)):
        change_str = f"{change_pct:.2f}% vs {year-1}"
    else:
        change_str = "N/A"
    row = df_pivot.iloc[row_idx]
    code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
    groupe = row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"]
    ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
    text = (
        f"[TYPE: ANNUAL_SUMMARY]\n"
        f"Résidence: {residence_label}\n"
        f"Ligne: {ligne}\n"
        f"Année: {year}\n"
        f"Contexte utilisé: {ctx or 'N/A'}\n"
        f"Valeur {year}: {fmt(val_cur)}\n"
        f"Valeur {year-1}: {fmt(val_prev)}\n"
        f"Variation: {change_str}\n"
        f"Code Hiérarchique: {code_h}\n"
        f"Groupe: {groupe}\n"
    )
    metadata = {
        "chunk_type": "ANNUAL_SUMMARY",
        "residence": residence_label,
        "year": year,
        "groupe": groupe,
        "code_h": code_h,
        "ligne": ligne,
        "row_index": int(df_pivot.index[row_idx])
    }
    return {"text": text, "metadata": metadata}

def make_trend_chunk(
    df_pivot: pd.DataFrame, row_idx: int, years: List[int],
    residence_label: str,
    contexte_preference: List[str] = ["Réel", "Prévision", "Budget"]):
    """
    Trend chunk over multiple years (type: TREND_MULTI_YEAR).
    years: list[int] e.g. [2022,2023,2024]
    """
    values = []
    used_ctx = None
    for y in years:
        v, ctx = get_value_for_year(df_pivot, row_idx, y, contexte_preference)
        if pd.notna(v) and safe_isfinite(v):
            values.append(v)
        else:
            values.append(None)
        if used_ctx is None and ctx is not None:
            used_ctx = ctx
    row = df_pivot.iloc[row_idx]
    code_h = row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"]
    groupe = row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"]
    ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
    def fmt(v):
        return f"{v:.0f} €" if v is not None and safe_isfinite(v) else "N/A"
    vals_str = ", ".join([f"{y}: {fmt(v)}" for y, v in zip(years, values)])
    first = values[0] if values else None
    last = values[-1] if values else None
    pct_total = None
    if first not in (None, 0) and last is not None and safe_isfinite(first) and safe_isfinite(last):
        try:
            pct_total = (last - first) / abs(first) * 100
        except Exception:
            pct_total = None
    pct_str = f"{pct_total:.2f}% ({years[0]}→{years[-1]})" if pct_total is not None and safe_isfinite(pct_total) else "N/A"
    text = (
        f"[TYPE: TREND_MULTI_YEAR]\n"
        f"Résidence: {residence_label}\n"
        f"Ligne: {ligne}\n"
        f"Années: {years}\n"
        f"Contexte principal trouvé: {used_ctx or 'N/A'}\n"
        f"Valeurs: {vals_str}\n"
        f"Variation globale: {pct_str}\n"
        f"Code Hiérarchique: {code_h}\n"
        f"Groupe: {groupe}\n"
    )
    metadata = {
        "chunk_type": "TREND_MULTI_YEAR",
        "residence": residence_label,
        "years": years,
        "groupe": groupe,
        "code_h": code_h,
        "ligne": ligne,
        "row_index": int(df_pivot.index[row_idx])
    }
    return {"text": text, "metadata": metadata}

def make_recommendation_chunk(
    df_pivot: pd.DataFrame,
    row_idx: int,
    years: List[int],
    residence_label: str,
    contexte_preference: List[str] = None):
    """
    Template de recommandations automatiques (type: RECOMMENDATION) basé sur trend simple.
    Peut être enrichi par des heuristiques supplémentaires.
    Utilise le chunk tendance calculé pour robustesse et éventuelle extension.
    """
    if contexte_preference is None:
        contexte_preference = ["Réel", "Prévision", "Budget"]

    # Récupération du chunk de tendance pour contexte et robustesse
    trend = make_trend_chunk(df_pivot, row_idx, years, residence_label, contexte_preference)
    # On va utiliser l'analyse de la tendance issue du chunk trend
    # On exploite pct_total si possible

    trend_metadata = trend.get("metadata", {})
    values = []
    for y in years:
        v, _ = get_value_for_year(df_pivot, row_idx, y, contexte_preference)
        values.append(v if (pd.notna(v) and safe_isfinite(v)) else None)
    first = values[0] if values else None
    last = values[-1] if values else None

    # Essaie d'utiliser le pourcentage trend du chunk trend, sinon recalcule localement
    pct_total = None
    trend_text = trend.get("text", "")
    try:
        # Recherche d'une ligne contenant "Variation globale: x%"
        import re
        m = re.search(r"Variation globale:\s*([-\d.,]+)%", trend_text)
        if m:
            pct_total = float(m.group(1).replace(',', '.'))
    except Exception:
        pct_total = None

    if pct_total is None and first not in (None, 0) and last is not None and safe_isfinite(first) and safe_isfinite(last):
        try:
            pct_total = (last - first) / abs(first) * 100
        except Exception:
            pct_total = None

    rec = []
    if pct_total is not None and safe_isfinite(pct_total):
        if pct_total > 10:
            rec.append(
                "Tendance fortement positive → vérifier si le budget sous-estime la croissance ; envisager de réallouer le budget investissement."
            )
        elif pct_total < -10:
            rec.append(
                "Tendance négative → investiguer les causes (baisse du CA / hausse des charges). Prioriser les postes 'Energie' et 'Maintenance'."
            )
        else:
            rec.append("Tendance stable → maintenir le monitoring mensuel.")
    else:
        rec.append("Données insuffisantes pour recommandations automatiques.")

    text = (
        f"[TYPE: RECOMMENDATION]\n"
        f"Résidence: {residence_label}\n"
        f"Années examinées: {years}\n"
        f"Variation détectée (extrait trend): "
        f"{pct_total:.1f}%\n" if pct_total is not None and safe_isfinite(pct_total) else
        f"Variation détectée: N/A\n"
    )
    text += f"Recommandations:\n- " + "\n- ".join(rec) + "\n"

    metadata = {
        "chunk_type": "RECOMMENDATION",
        "residence": residence_label,
        "years": years,
        "row_index": int(df_pivot.index[row_idx]),
        "trend_metadata": trend_metadata
    }
    return {"text": text, "metadata": metadata}

# ---------------------------
# Pipeline principal : génération de tous les chunks
# ---------------------------
def generate_chunks_from_pivot(
    df_pivot: pd.DataFrame,
    residence_label: str,
    years_for_trend: List[int] = None,
    contexte_preference: List[str] = None) -> pd.DataFrame:
    """
    Parcourt chaque ligne (Ligne_Analytique) du df_pivot et génère :
        - ligne-cell chunks pour chaque year in columns (Annuelle)
        - annual summary chunks
        - trend chunks (par défaut pour dernières 3 années trouvées)
        - recommendation chunk
    Retourne un DataFrame chunks_df avec colonnes: text, metadata (dict)
    """
    if years_for_trend is None:
        years = sorted({c[0] for c in df_pivot.columns if isinstance(c, tuple) and isinstance(c[0], (int, np.integer))})
        years_for_trend = years[-3:]
    if contexte_preference is None:
        contexte_preference = ["Réel", "Prévision", "Budget"]

    chunks: List[Dict[str, Any]] = []
    n_rows = df_pivot.shape[0]
    for i in range(n_rows):
        row = df_pivot.iloc[i]
        ligne = row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"]
        if (isinstance(ligne, str) and ligne.strip() == "") or (not isinstance(ligne, str) and (pd.isna(ligne) if not isinstance(ligne, pd.Series) else ligne.isna().all())):
            continue
        years_all = sorted({c[0] for c in df_pivot.columns if isinstance(c, tuple) and c[3] == "Annuelle" and isinstance(c[0], (int, np.integer))})
        for y in years_all:
            chunks.append(make_line_cell_chunk(df_pivot, i, y, residence_label, contexte_preference))
            chunks.append(make_annual_summary_chunk(df_pivot, i, y, residence_label, contexte_preference))
        if len(years_for_trend) >= 2:
            chunks.append(make_trend_chunk(df_pivot, i, years_for_trend, residence_label, contexte_preference))
            chunks.append(make_recommendation_chunk(df_pivot, i, years_for_trend, residence_label))

    chunks_df = pd.DataFrame([{
        "text": c["text"],
        **{f"meta_{k}": (v if not isinstance(v, (list, dict)) else str(v)) for k, v in c["metadata"].items()}
    } for c in chunks])
    return chunks_df

# ---------------------------
# Utilisation (exemple)
# ---------------------------
# df_pivot_sorted : ton DataFrame pivot affiché (index numérique, colonnes MultiIndex)
# residence_label : "11000099 - Noisy" (ou simplement "Noisy")
# years_for_trend : par défaut dernières 3 années disponibles

# Exemple d'appel :
chunks_df = generate_chunks_from_pivot(df_pivot_sorted, residence_label="Noisy")
chunks_df.to_csv("chunks_from_pivot.csv", index=False)

NameError: name 'nature_ecriture' is not defined

In [273]:
sorted(set([col[3] for col in df_pivot.columns[3:] if isinstance(col, tuple)]))

['Annuelle', 'Mensuelle']

In [274]:
chunks_df.shape

(1812, 11)

In [275]:
chunks_df.columns

Index(['text', 'meta_chunk_type', 'meta_residence', 'meta_year',
       'meta_contexte', 'meta_groupe', 'meta_code_h', 'meta_ligne',
       'meta_row_index', 'meta_years', 'meta_trend_metadata'],
      dtype='object')

In [276]:
print(chunks_df.loc[
    (chunks_df["meta_ligne"] == "% DES RECETTES TOTALES") & 
    (chunks_df["meta_chunk_type"] == "LINE_CELL"),
    "text"
].iloc[0])

[TYPE: LINE_CELL]
Résidence: Noisy
Année: 2022
Contexte: Réel
Groupe: Chiffre d'affaire
Code Hiérarchique: 4.
Ligne analytique: % DES RECETTES TOTALES
Valeur annuelle: N/A



---

In [ ]:
# Je veux cette liste de colonnes
annee = sorted(set([col[0] for col in df_pivot_sorted.columns[3:] if isinstance(col, tuple)]))
annee

[2022, 2023, 2024, 2025, 2026]

In [203]:
for i, row in df_pivot_sorted.iterrows():
    print(row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"])
    if i == 5:
        break

Chiffre d'affaire
Chiffre d'affaire
Chiffre d'affaire
Chiffre d'affaire
Chiffre d'affaire
Chiffre d'affaire


In [177]:
chunks = []

In [220]:
# ------------------------
# 1. CHUNKS PAR LIGNE
# ------------------------
chunks_by_lines = []
annees = sorted(set([col[0] for col in df_pivot_sorted.columns[3:] if isinstance(col, tuple)]))
for _, row in df_pivot_sorted.iterrows():
    doc = {
        "type": "ligne",
        "groupe": row["Groupe"].iloc[-1] if hasattr(row["Groupe"], "iloc") else row["Groupe"],
        "code_h": row["Code_H"].iloc[-1] if hasattr(row["Code_H"], "iloc") else row["Code_H"],
        "ligne": row["Ligne_Analytique"].iloc[-1] if hasattr(row["Ligne_Analytique"], "iloc") else row["Ligne_Analytique"],
        "valeurs": {str(a): row.get(str(a), None) for a in annee}
    }
    chunks_by_lines.append(doc)

pd.DataFrame(chunks_by_lines)

,type,groupe,code_h,ligne,valeurs
0,ligne,Chiffre d'affaire,1.,RECETTES,"{'2022': None, '2023': None, '2024': None, '20..."
1,ligne,Chiffre d'affaire,1.1.,Loyers logements et parkings HT,"{'2022': None, '2023': None, '2024': None, '20..."
2,ligne,Chiffre d'affaire,1.1.1.,CA Locatif Estudines,"{'2022': None, '2023': None, '2024': None, '20..."
3,ligne,Chiffre d'affaire,1.1.4.,CA Locatif Parkings,"{'2022': None, '2023': None, '2024': None, '20..."
4,ligne,Chiffre d'affaire,1.2.,RECETTES ANNEXES,"{'2022': None, '2023': None, '2024': None, '20..."
...,...,...,...,...,...
146,ligne,Chiffre d'affaire,10.,% DES RECETTES TOTALES,"{'2022': None, '2023': None, '2024': None, '20..."
147,ligne,Chiffre d'affaire,11.,Dotations aux amortissements (CAPEX),"{'2022': None, '2023': None, '2024': None, '20..."
148,ligne,Marge,12.,EBITDA,"{'2022': None, '2023': None, '2024': None, '20..."
149,ligne,Charge,13.,CAPEX,"{'2022': None, '2023': None, '2024': None, '20..."


In [ ]:
# ------------------------
# 2. CHUNKS PAR GROUPE
# ------------------------
chunks_by_group = []
for group_name, group_df_pivot_sorted in df_pivot_sorted.groupby("Groupe"):
    doc = {
        "type": "groupe",
        "groupe": group_name,
        "lignes": []
    }
    for _, row in group_df_pivot_sorted.iterrows():
        doc["lignes"].append(
            {
                "code_h": row["Code_H"],
                "ligne": row["Ligne_Analytique"],
                "2022": row.get("2022"),
                "2023": row.get("2023"),
                "2024": row.get("2024"),
                "2025_budget": row.get("2025"),
                "2025_prevision": row.get("Prévision"),
                "2026_budget": row.get("2026"),
            }
        )
    chunks_by_group.append(doc)
chunks_by_group

In [ ]:
# ------------------------
# 3. CHUNKS PAR ANNÉE
# ------------------------
chunks_by_year = []
annees = sorted(set([col[0] for col in df_pivot_sorted.columns[3:] if isinstance(col, tuple)]))
for an in annees:
    cols = [c for c in df_pivot_sorted.columns if c.startswith(str(an))]
    if not cols:
        continue

    doc = {
        "type": "année",
        "annee": an,
        "valeurs": []
    }

    for _, row in df_pivot_sorted.iterrows():
        if an in row.index:
            doc["valeurs"].append({
                "groupe": row["Groupe"],
                "code_h": row["Code_H"],
                "ligne": row["Ligne_Analytique"],
                "valeur": row.get(an),
            })

    chunks_by_year.append(doc)
chunks_by_year

---

In [77]:
def create_document_content(row):
    """
    Crée le contenu textuel riche pour l'embedding
    """
    content_parts = []
    
    content_parts.append(f"Résidence: {row['Section  analytique']}")
    content_parts.append(f"Code_h: {row['Code Hiérarchique']}")
    content_parts.append(f"Libellé ligne: {row['Lignes']}")
    if row['Contexte'] == "R":
        content_parts.append("Contexte: Réel")
    elif row['Contexte'] == "B":
        content_parts.append("Contexte: Budget")
    elif row['Contexte'] == "P":
        content_parts.append("Contexte: Prévison")
    content_parts.append(f"Montant: {row['Montant']}€")
    content_parts.append(f"Nature écriture: {row['Nature de l\'écriture']}")
    content_parts.append(f"Mois: {int(row['Mois'])}")
    content_parts.append(f"Année: {int(row['Année'])}")
    content_parts.append(f"Groupe: {row['Groupe']}")
    
    return " | ".join(content_parts)

#==================================================================
#==================================================================

for _, row in df.iloc[:2].iterrows():
    print(create_document_content(row))
    print("-"*100)

Résidence: 11000099 - Noisy | Code_h: 4 | Libellé ligne: % DES RECETTES TOTALES | Contexte: Réel | Montant: 95€ | Nature écriture: Mensuelle | Mois: 11 | Année: 2022 | Groupe: Chiffre d'affaire
----------------------------------------------------------------------------------------------------
Résidence: 11000099 - Noisy | Code_h: 7 | Libellé ligne: % DES RECETTES TOTALES | Contexte: Prévison | Montant: 87€ | Nature écriture: Mensuelle | Mois: 8 | Année: 2025 | Groupe: Chiffre d'affaire
----------------------------------------------------------------------------------------------------


In [79]:
def create_metadata(row):
    """
    Crée les métadonnées pour le filtrage
    """
    metadata = {
        "annee": int(row['Année']),
        "mois": int(row['Mois']),
        "contexte": (
            "Réel" if row["Contexte"] == "R"
            else "Budget" if row["Contexte"] == "B"
            else "Prévision" if row["Contexte"] == "P"
            else str(row["Contexte"])
        ),
        "type_contexte": row["Colonnes"],
        "groupe": row.get("Groupe", "Non spécifié"),
        "residence": row["Section  analytique"],
        "montant": float(row["Montant"]),
        "nature": row["Nature de l'écriture"]
    }
    
    if pd.notna(row.get('Code Hiérarchique')):
        metadata["code_hierarchique"] = str(row['Code Hiérarchique'])
    
    return metadata

#==================================================================
#==================================================================

for _, row in df.iloc[:2].iterrows():
    print(create_metadata(row))
    print("-"*100)

{'annee': 2022, 'mois': 11, 'contexte': 'Réel', 'type_contexte': 'Réel 2022', 'groupe': "Chiffre d'affaire", 'residence': '11000099 - Noisy', 'montant': 95.0, 'nature': 'Mensuelle', 'code_hierarchique': '4'}
----------------------------------------------------------------------------------------------------
{'annee': 2025, 'mois': 8, 'contexte': 'Prévision', 'type_contexte': 'Prév 2025', 'groupe': "Chiffre d'affaire", 'residence': '11000099 - Noisy', 'montant': 87.0, 'nature': 'Mensuelle', 'code_hierarchique': '7'}
----------------------------------------------------------------------------------------------------


In [97]:
def create_embeddings_context(row):
    """
    Crée un contexte enrichi pour de meilleurs embeddings
    """
    context = []
    
    # Combinaison intelligente des champs
    context.append(f"{row['Colonnes']} pour {row['Section  analytique']}")
    
    if pd.notna(row.get('Lignes')):
        context.append(f"Ligne: {row['Lignes']}")
    
    if pd.notna(row.get('Groupe')):
        context.append(f"Catégorie: {row['Groupe']}")
        
    context.append(f"{row["Nature de l'écriture"]}")

    month_name = [
        "Janvier", "Février", "Mars", "Avril", "Mai", "Juin",
        "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre"
    ]
    if row["Nature de l'écriture"] == "Annuelle":
        annee_int = int(row['Année'])
        context.append(f"En {annee_int}")
    else:
        mois_int = int(row['Mois'])
        annee_int = int(row['Année'])
        if 1 <= mois_int <= 12:
            mois_nom = month_name[mois_int - 1]
        else:
            mois_nom = str(mois_int)
        context.append(f"En {mois_nom} {annee_int}")
    
    context.append(f"Montant: {row['Montant']}€")
    
    return " - ".join(context)

#==================================================================
#==================================================================

for _, row in df.iloc[37:42].iterrows():
    print(create_embeddings_context(row))
    print("-"*100)

Budget 2025 pour 11000099 - Noisy - Ligne: % DES RECETTES TOTALES - Catégorie: Chiffre d'affaire - Mensuelle - En Octobre 2025 - Montant: 73€
----------------------------------------------------------------------------------------------------
Budget 2026 pour 11000099 - Noisy - Ligne: % DES RECETTES TOTALES - Catégorie: Chiffre d'affaire - Annuelle - En 2026 - Montant: 73€
----------------------------------------------------------------------------------------------------
Budget 2026 pour 11000099 - Noisy - Ligne: % DES RECETTES TOTALES - Catégorie: Chiffre d'affaire - Mensuelle - En Août 2026 - Montant: 73€
----------------------------------------------------------------------------------------------------
Budget 2025 pour 11000099 - Noisy - Ligne: % DES RECETTES TOTALES - Catégorie: Chiffre d'affaire - Mensuelle - En Février 2025 - Montant: 72€
----------------------------------------------------------------------------------------------------
Budget 2026 pour 11000099 - Noisy - Lign

In [98]:
def create_rag_lists_from_df(df):
    """
    Transforme les données tabulaires en plusieurs listes parallèles (texte, métadonnées, contextes d'embedding, ids)
    """
    texts = []
    metadatas = []
    contexts = []
    ids = []
    for _, row in df.iterrows():
        # Génération du contenu, des métadonnées et du contexte
        text = create_document_content(row)
        meta = create_metadata(row)
        ctx = create_embeddings_context(row)
        id_ = f"{row['Section  analytique']}_{row['Lignes']}_{row['Colonnes']}"
        
        texts.append(text)
        metadatas.append(meta)
        contexts.append(ctx)
        ids.append(id_)
    return {
        "ids": ids,
        "texts": texts,
        "metadatas": metadatas,
        "contexts": contexts
    }

In [ ]:
# Recherche : renvoie les indices et similarités, plus simple maintenant
def search_similar_texts(query, texts, contexts, embedding_model, top_k=5):
    """
    Recherche des textes similaires (parmi les contexts ou les texts selon le use-case)
    """
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    model = SentenceTransformer(embedding_model)

    # Embedding de la requête
    query_embedding = model.encode([query], normalize_embeddings=True)

    # Embeddings des contexts
    contexts_embeddings = model.encode(contexts, normalize_embeddings=True)

    # Calcul des similarités
    similarities = cosine_similarity(query_embedding, contexts_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    results = []
    for idx in top_indices:
        results.append({
            'index': idx,
            'text': texts[idx],
            'similarity': float(similarities[idx])
        })
    return results

# Exécution principale minimaliste
rag_data = create_rag_lists_from_df(df)
print(f"Créé {len(rag_data['texts'])} textes pour le RAG")

# Exemple de recherche
query = "Quelle est la recette annuelle réelle de 2023 ?"
results = search_similar_texts(
    query, 
    rag_data["texts"], 
    rag_data["contexts"], 
    embedding_model='all-MiniLM-L6-v2', 
    top_k=3
)

for i, result in enumerate(results):
    print(f"\nRésultat {i+1} (similarité: {result['similarity']:.3f}):")
    print(result['text'][:200] + "...")

Créé 7522 textes pour le RAG

Résultat 1 (similarité: 0.637):
Résidence: 11000099 - Noisy | Code_h: 10 | Libellé ligne: % DES RECETTES TOTALES | Contexte: Réel | Montant: -9€ | Nature écriture: Mensuelle | Mois: 12 | Année: 2023 | Groupe: Chiffre d'affaire...

Résultat 2 (similarité: 0.637):
Résidence: 11000099 - Noisy | Code_h: 10 | Libellé ligne: % DES RECETTES TOTALES | Contexte: Réel | Montant: 31€ | Nature écriture: Mensuelle | Mois: 6 | Année: 2023 | Groupe: Chiffre d'affaire...

Résultat 3 (similarité: 0.635):
Résidence: 11000099 - Noisy | Code_h: 10 | Libellé ligne: % DES RECETTES TOTALES | Contexte: Réel | Montant: -23€ | Nature écriture: Mensuelle | Mois: 11 | Année: 2023 | Groupe: Chiffre d'affaire...


# Vectorisation et enregistrement dans chromadb

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

d:\SL_CB_IA\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
EMBEDDING_MODEL_NAME = 'distiluse-base-multilingual-cased-v2' 
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [ ]:
ids = [f"doc_{i}" for i in range(len([]))]

In [ ]:
def clean_metadata_types(metadata_list):
    """
    Convertit récursivement les types NumPy (int64, float64, etc.) 
    en types Python natifs (int, float) dans une liste de dictionnaires.
    Ceci est essentiel pour la compatibilité avec ChromaDB.
    """
    cleaned_metadata = []
    for entry in metadata_list:
        cleaned_entry = {}
        for key, value in entry.items():
            # Conversion des types numériques NumPy
            if isinstance(value, (np.int64, np.int32, np.int8)):
                cleaned_entry[key] = int(value)
            elif isinstance(value, (np.float64, np.float32)):
                # Utiliser round() ou s'assurer que float(value) n'est pas NaN
                cleaned_entry[key] = float(value) if not np.isnan(value) else None
            elif isinstance(value, (int, float, str, bool, type(None))):
                cleaned_entry[key] = value
            else:
                # Gérer d'autres types non supportés si nécessaire (ex: convertir en str)
                cleaned_entry[key] = str(value)
        cleaned_metadata.append(cleaned_entry)
    return cleaned_metadata

new_metadata = clean_metadata_types(new_metadata)

In [ ]:
def generate_embeddings(texts):
    """Génère les vecteurs à partir du texte."""
    # Note : Le calcul peut être long si le nombre de chunks est très élevé (milliers)
    return embed_model.encode(texts).tolist()

embeddings = generate_embeddings(new_chunks)

In [ ]:
def indexer_chromadb(embeddings, chunks, metadata, ids):
    """Indexe les chunks dans ChromaDB."""
    try:
        # --- Configuration ChromaDB ---
        client = chromadb.Client()
        collection_name = 'residence_finance_analysis_complete'

        # Suppression pour un ré-indexage propre
        try:
            client.delete_collection(name=collection_name)
        except Exception:
            pass # Ignore si la collection n'existe pas

        collection = client.get_or_create_collection(
            name=collection_name, 
            metadata={"hnsw:space": "cosine"}
        )

        # Ajout des documents à la collection
        collection.add(
            embeddings=embeddings,
            documents=chunks,
            metadatas=metadata,
            ids=ids
        )

        print(f"\n✅ Indexation complète. {collection.count()} documents analytiques sont maintenant disponibles dans ChromaDB.")
        print("La base est prête pour l'interrogation par Ollama via RAG.")
    except Exception as e:
        print(f"Erreur lors de l'indexation: {e}")

# Lancement de l'indexation
indexer_chromadb(embeddings, new_chunks, new_metadata, ids)


✅ Indexation complète. 4135 documents analytiques sont maintenant disponibles dans ChromaDB.
La base est prête pour l'interrogation par Ollama via RAG.
